In [197]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [198]:
# load data
tree_data = pd.read_csv('data/tree.csv')
genes_data = pd.read_csv('data/vert_genes.csv')

print(tree_data)
print(genes_data)


     Parent  Child     age_ch          t                 species
0     222.0      1   0.000000   9.000250  Peromyscus_maniculatus
1     222.0      2   0.000000   9.000250            Mus_musculus
2     221.0      3   0.000000  12.172706      Cricetulus_griseus
3     220.0      4   0.000000  14.684269       Rattus_norvegicus
4     219.0      5   0.000000  17.062881    Mesocricetus_auratus
..      ...    ...        ...        ...                     ...
402   399.0    403   7.861905   1.640772                     NaN
403   403.0    404   5.750020   2.111885                     NaN
404   404.0    405   5.500000   0.250020                     NaN
405   403.0    406   5.000000   2.861905                     NaN
406     NaN    407  50.024697   2.091292                     NaN

[407 rows x 5 columns]
        ensembl_id orthId  glength                       species
0     XP_009078534  1CPN2    66213          Acanthisitta_chloris
1     XP_005013863  1CPN2    44913            Anas_platyrhynchos
2

In [199]:
class Node:
    def __init__(self,data):
        self.l = None
        self.r = None
        self.data = data


In [200]:
def create_tree(root):
    children = tree_data[tree_data['Parent']==root]
    node = Node(tree_data[tree_data['Child']==root])
    if len(children)>0:
        for i in range(0, len(children)):
            if i == 0:
                node.l = create_tree(children.iloc[1].iloc[1])
            else:
                node.r = create_tree(children.iloc[0].iloc[1])
        return node
    else:
        return node



In [201]:
# gamma_0 + (alpha * time) (beta * parent gene len) + (gamma * time * parent gene len)

# Parameters
# for Z_0
alpha_0 = 50000
variance_0 = 5000

# for Z_i and X_i
alpha = 0
beta = 1
variance = 2500 



In [210]:
tree = create_tree(407)

mean = np.zeros(407)
covariance = np.array(407*[np.zeros(407)])

mean[406] = alpha_0
covariance[406,406] = variance_0

def meanCovariance(tree, mean, covariance):
    if tree != None:
        index = int(tree.data.iloc[0].iloc[1])
        t = int(tree.data.iloc[0].iloc[3])
        parent = int(tree.data.iloc[0].iloc[0])
        mean[index-1] = mean[parent-1]*beta+alpha*t

        covariance[index-1,:] = covariance[parent-1,:]*beta 
        covariance[:,index-1] = covariance[parent-1,:]*beta 

        covariance[index-1,index-1] = variance*t+beta*covariance[parent-1,parent-1]
        '''covariance[index-1,parent-1] = beta**2*covariance[parent-1,parent-1]
        covariance[parent-1,index-1] = beta**2*covariance[parent-1,parent-1]'''
        meanCovariance(tree.l,mean,covariance)
        meanCovariance(tree.r,mean,covariance)
    else:
        return

meanCovariance(tree.l,mean,covariance)
meanCovariance(tree.r,mean,covariance)

print(covariance)
print(mean)


[[112500.  90000.  82500. ...   5000.   5000.   5000.]
 [ 90000. 112500.  82500. ...   5000.   5000.   5000.]
 [ 82500.  82500. 112500. ...   5000.   5000.   5000.]
 ...
 [  5000.   5000.   5000. ... 110000. 105000.   5000.]
 [  5000.   5000.   5000. ... 105000. 110000.   5000.]
 [  5000.   5000.   5000. ...   5000.   5000.   5000.]]
[50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000.
 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000.
 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000.
 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000.
 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000.
 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000.
 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000.
 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000.
 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000. 50000.
 50000. 50000. 50000. 500

In [211]:
#fik lige en bug her så det er kommenteret ud, men bruger det heller ikke lige pt. skal dog bruge det til message passing
'''J = np.linalg.inv(covariance)
h = np.matmul(J,mean)
print(J)
print(h)'''

LinAlgError: Singular matrix

In [ ]:
#load gene data og få lagt det ind i arrays, hvis et gene ikke er der sættes dens længde til 0.
a = np.array(10*[np.zeros(204)])

unique_genes = genes_data.loc[:,'orthId'].unique()

for i in range(0,204):
    species = tree_data.iloc[i].iloc[4]
    extract = genes_data[genes_data['species']==species]
    for j in range(0,10):
        temp = extract[extract['orthId']==unique_genes[j]]
        if len(temp)==1:
            a[j,i] = temp['glength']

print(a)
print(a.shape)

[[ 88338. 219992.  89972. ...   4568.   4239.  66937.]
 [ 30935.  31081.  40971. ...  34954.  33799.  31589.]
 [ 21555.  20662.  12707. ...   8561.   8475.      0.]
 ...
 [ 19977.  21211.  24187. ...   3964.      0.      0.]
 [ 16494.  15176.  16651. ...   7387.  19106.  18206.]
 [ 29924.   8341.   6260. ...   1881.   1852.   1778.]]
(10, 204)


In [214]:
#estimere P(Z_i|x_1,..,x_204) og print normalfordelingen med (mean,variance)
for j in range(204,407):
    sigmaYX = covariance[j,:204]
    sigmaXY = covariance[:204,j]
    sigmaXX = np.linalg.inv(covariance[:204,:204])
    meanY = mean[j]+np.matmul(np.matmul(sigmaYX,sigmaXX),a[0]-mean[:204])
    covarianceY = covariance[j,j]-np.matmul(np.matmul(sigmaYX,sigmaXX),sigmaXY)
    print((meanY,covarianceY))


(57537.211454067285, 4860.499557893132)
(81129.12986366001, 6836.0091792402545)
(97231.85672496926, 7534.0082883548985)
(117623.18157348582, 7285.046341063515)
(124382.33710097564, 7115.295733204199)
(140738.885439905, 7295.260813949048)
(166132.4997069363, 6824.046447462097)
(189398.66410301332, 5176.655410902444)
(194387.9505748283, 6360.6673145461245)
(199109.07486503987, 6666.667877939566)
(204964.92684880906, 7202.039723246438)
(209460.93298048276, 7181.416891535591)
(209435.32443677512, 7157.154011101389)
(205576.32872429083, 7193.2427468517635)
(200057.78866699504, 7046.001903933167)
(188565.92962934636, 6833.398433072929)
(170701.3462530323, 6814.526008526562)
(164086.80775181955, 6953.229363069404)
(233179.84253326213, 5073.099315237865)
(231511.6556969375, 4457.251924635726)
(237878.36368746698, 3458.4446737580583)
(237878.36368746698, 3458.4446737580583)
(189398.66410301332, 5176.655410902444)
(192985.93629987555, 5433.572623730448)
(202683.07754611975, 6150.799003680855)
(2

In [215]:
#et forsøg på at lave det med j og h - tror dog ikke det er nødvendigt at gå den vej

'''for j in range(204,205):
    hy = h[j]
    print(hy)
    Jyx = J[j,:204]
    hJ = hy-np.matmul(Jyx,a[0])
    Jyy = covariance[j,j]
    print(hJ)
    print(Jyy)
    varianceY = np.linalg.inv(Jyy)
    print(varianceY)
    meanY = np.matmul(hJ,varianceY)    
    print(meanY)
'''

'for j in range(204,205):\n    hy = h[j]\n    print(hy)\n    Jyx = J[j,:204]\n    hJ = hy-np.matmul(Jyx,a[0])\n    Jyy = covariance[j,j]\n    print(hJ)\n    print(Jyy)\n    varianceY = np.linalg.inv(Jyy)\n    print(varianceY)\n    meanY = np.matmul(hJ,varianceY)    \n    print(meanY)\n'